# Comparaciones entre algoritmos

## Importación de librerías

In [ ]:
import math
import numpy as np
import random
from scipy.linalg import hadamard
from google.colab import files
import pandas as pd
import time
import openpyxl
from openpyxl import Workbook
from numba import jit

## Count Mean Sketch

In [ ]:
class CountMeanSketch:

  # Constructor de la clase
  def __init__(self, epsilon, k, m):

    self.epsilon = epsilon
    self.k = k
    self.m = m

    # Se crea una familia de d funciones a partir del método hash_function, que por su construcción garantiza que la familia sea 3-independiente
    self.hash_family = [self.hash_function() for _ in range(k)]

    # Se crea la matriz sketch M y se inicializa en cero
    self.M = np.zeros((k, m))


  # Lado cliente
  def Cliente(self, dataset):

    self.res_cliente = []

    for d in dataset:

      # Se elige j al azar
      j = random.randint(0, self.k - 1)

      # Se inicia vector v en -1 de tamaño m
      v = np.full(self.m, -1)

      # Se coloca un 1 en la posición v[h_j(d)]
      v[self.hash_family[j](d)] = 1

      # Se genera un vector b
      b = self.generar_b()

      # Se calcula el vector v'
      v2 = v * b

      # Se acumulan los pares en la lista res_cliente
      self.res_cliente.append([v2, j])


  # Lado servidor
  def Servidor(self):

    # Se define c_epsilon
    c = (math.exp(self.epsilon/2) + 1) / (math.exp(self.epsilon/2) - 1)

    for v2, j in self.res_cliente: # se hace para cada par de elementos recibidos del lado cliente
      x = self.k * (c/2 * v2 + 1/2)

      # Se actualiza la matriz M
      self.M[j, :] += x # Se suma a la fila j cada valor del vector x



  # Se calcula la frecuencia de un elemento en el dataset
  def frecuencia(self, element):

    # Se guarda el valor de cada fila de la matriz correspondiente a la aplicación de la función hash sobre el elemento
    return (self.m/(self.m - 1)) * (1/self.k * sum([self.M[l, self.hash_family[l](element)] for l in range(self.k)]) - len(self.res_cliente)/self.m)



  # Este método devuelve una función hash
  # Técnica propuesta por Carter y Wegman empleando polinomios con coeficiente aleatorios
  def hash_function(self):

    # Se elige un número primo grande aleatorio p, por ejemplo, el siguiente
    p = 2**61 - 1

    # Se eligen dos enteros a y b de forma aleatoria entre 0 y el número primo p
    a = random.randint(0, p)
    b = random.randint(0, p)
    c = random.randint(0, p)

    # Se devuelve una función que dado un elemento devuelve un entero entre 0 y m-1
    return lambda element: (a * abs(hash(element))**2 + b * abs(hash(element)) + c) % p % self.m

  # Método que genera un vector b de 1s y -1s
  def generar_b(self): # se le pasa el tamaño del vector

    b_l = [1, -1]

    p1 = math.exp(self.epsilon/2) / (1 + math.exp(self.epsilon/2))
    p2 = 1 / (1 + math.exp(self.epsilon/2))

    b = random.choices(b_l, weights = [p1, p2], k = self.m)

    return np.array(b)

## Hadamard Count Mean Sketch

In [ ]:
class HadamardCountMeanSketch:

  # Constructor de la clase
  def __init__(self, epsilon, k, m):

    # Comprueba si m es potencia de 2 (necesario para la matriz de Hadamard)
    if m <= 0 or not(math.log2(m).is_integer()):
        raise ValueError("El valor de m debe ser potencia de 2.")

    self.epsilon = epsilon
    self.k = k
    self.m = m

    # Se crea una familia de d funciones a partir del método hash_function, que por su construcción garantiza que la familia sea 3-independiente
    self.hash_family = [self.hash_function() for _ in range(k)]

    # Se crea la matriz sketch M y se inicializa en cero
    self.M = np.zeros((k, m))


  # Lado cliente
  def Cliente(self, dataset):

    self.res_cliente = []

    hadamard_matrix = hadamard(self.m)

    for d in dataset:

      # Se elige j al azar
      j = random.randint(0, self.k - 1)

      # Se inicia vector v en 0 de tamaño m
      v = np.full(self.m, 0)

      # Se coloca un 1 en la posición v[h_j(d)]
      v[self.hash_family[j](d)] = 1

      # Se calcula w (el operador @ multiplica matrices)
      w = hadamard_matrix @ v

      # Se elige l al azar
      l = random.randint(0, self.m - 1)

      # Se genera un vector b
      b = self.generar_b()

      # Se calcula w' (el resultado es un escalar)
      w2 = b * w[l]

      # Se acumulan los pares en la lista res_cliente
      self.res_cliente.append([w2, j, l])


  # Lado servidor
  def Servidor(self):

    # Se define c_epsilon
    c = (math.exp(self.epsilon) + 1) / (math.exp(self.epsilon) - 1)

    for w2, j, l in self.res_cliente: # se hace para cada trío de elementos recibidos del lado cliente
      x = self.k * c * w2

      # Se actualiza la matriz M
      self.M[j, l] += x # Se suma a la fila j y columna l cada valor del vector x

    self.M = self.M @ np.transpose(hadamard(self.m))


  # Se calcula la frecuencia de un elemento en el dataset
  def frecuencia(self, element):

    # Se guarda el valor de cada fila de la matriz correspondiente a la aplicación de la función hash sobre el elemento
    return (self.m/(self.m - 1)) * (1/self.k * sum([self.M[l, self.hash_family[l](element)] for l in range(self.k)]) - len(self.res_cliente)/self.m)



  # Este método devuelve una función
  def hash_function(self):

    # Se elige un número primo grande aleatorio p, por ejemplo, el siguiente
    p = 2**61 - 1

    # Se eligen dos enteros a y b de forma aleatoria entre 0 y el número primo p
    a = random.randint(0, p)
    b = random.randint(0, p)
    c = random.randint(0, p)

    # Se devuelve una función que dado un elemento devuelve un entero entre 0 y m-1
    return lambda element: (a * abs(hash(element))**2 + b * abs(hash(element)) + c) % p % self.m

  # Método que genera un vector b de 1s y -1s
  def generar_b(self): # se le pasa el tamaño del vector

    b_l = [1, -1]

    p1 = math.exp(self.epsilon/2) / (1 + math.exp(self.epsilon/2))
    p2 = 1 / (1 + math.exp(self.epsilon/2))

    b = random.choices(b_l, weights = [p1, p2])

    return b[0]

## Comparación

In [ ]:
files.upload() # importar kaggle.json

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction
!unzip airline-passenger-satisfaction.zip

Dataset URL: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction
License(s): other
airline-passenger-satisfaction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  airline-passenger-satisfaction.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
data = pd.read_csv('train.csv')
data['Class'].value_counts()

Class
Business    49665
Eco         46745
Eco Plus     7494
Name: count, dtype: int64

In [ ]:
dataset = data['Class']

In [ ]:
def CMS(epsilon, k, m, dataset, elemento):

  inicio = time.time()

  cm = CountMeanSketch(epsilon, k, m)
  cm.Cliente(dataset)
  cm.Servidor()
  frecuencia = cm.frecuencia(elemento)

  fin = time.time()
  tiempo = fin - inicio

  error = abs(dataset.value_counts()[elemento] - frecuencia)  # diferencia absoluta

  coste_transmision = len(dataset) * m   # en bits
  coste_kB = coste_transmision / 8 / 1024 # kilobytes

  return tiempo, frecuencia, error, coste_kB

def HCMS(epsilon, k, m, dataset, elemento):

  inicio = time.time()

  cm = HadamardCountMeanSketch(epsilon, k, m)
  cm.Cliente(dataset)
  cm.Servidor()
  frecuencia = cm.frecuencia(elemento)

  fin = time.time()
  tiempo = fin - inicio

  error = abs(dataset.value_counts()[elemento] - frecuencia)  # diferencia absoluta

  # coste_transmision = m   # en bits (Creo que está mal ya que en HCMS se envía un bit por cada registro, no se envía el vector m)
  coste_transmision = len(dataset)
  coste_kB = coste_transmision / 8 / 1024 # kilobytes

  return tiempo, frecuencia, error, coste_kB

In [ ]:
wb = Workbook()

# Hacer el archivo de trabajo activo
ws = wb.active

# Añadir registro
ws.append(["Parámetros", "TiempoCMS", "TiempoHCMS", "FrecuenciaCMS", "FrecuenciaHCMS", "ErrorCMS", "ErrorHCMS", "CosteCMS", "CosteHCMS"])

In [ ]:
for k in [15, 16]: # opciones de k
  for m in [8, 10]: # opciones de m
    for e in [1, 2, 3]: # opciones de epsilon
      tiempo, frecuencia, error, coste = CMS(2**e, 2**k, 2**m, dataset, 'Business')
      tiempo2, frecuencia2, error2, coste2 = HCMS(2**e, 2**k, 2**m, dataset, 'Business')

      # añadimos filas por su nombre de fila
      fila = '(2^'+ str(e) + ', 2^'+ str(k) + ', 2^'+ str(m) + ')'
      ws.append([fila, tiempo, tiempo2, frecuencia, frecuencia2, error, error2, coste, coste2])

# Guardar el archivo en cualquier ruta, si no la especificas detalladamente la pondra en la carpeta donde estas corriendo este programa
wb.save("Comparaciones-CMS-HCMS.xlsx")